In [12]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: udacity-demo
Azure region: centralus
Subscription id: c5da1480-248e-4cc5-ba27-697bc9911ecf
Resource group: udacity-demo


In [13]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "demo-cluster"

compute_config = AmlCompute.provisioning_configuration(
    vm_size = "STANDARD_D2_V2",
    min_nodes=0,
    max_nodes=4,
)

try:
    clusterhandler = ComputeTarget(workspace=ws, name=cluster_name)
    print("Cluster is existing")
except:
    print("Could not find cluster")
    clusterhandler = ComputeTarget.create(workspace=ws, name=cluster_name, provisioning_configuration=compute_config)
clusterhandler.wait_for_completion()
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###


Cluster is existing


In [14]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    'max_iter':choice(10,50,200,300,500),
    'C':uniform(.02,1)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=3,slack_factor=0.1)  

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                compute_target=clusterhandler,
                script='train.py',
                environment=sklearn_env
                )

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                              hyperparameter_sampling=ps,
                              policy=policy,
                              primary_metric_name='Accuracy',
                              primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                              max_total_runs=20,
                              max_concurrent_runs=4)

print("hyperdrive config done")

hyperdrive config done


In [15]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run=exp.submit(config=hyperdrive_config)
hyperdrive_run.wait_for_completion(show_output=True)


RunId: HD_2f3a8400-3c2e-45b4-a870-764a376c95e0
Web View: https://ml.azure.com/runs/HD_2f3a8400-3c2e-45b4-a870-764a376c95e0?wsid=/subscriptions/c5da1480-248e-4cc5-ba27-697bc9911ecf/resourcegroups/udacity-demo/workspaces/udacity-demo&tid=db05faca-c82a-4b9d-b9c5-0f64b6755421

Streaming azureml-logs/hyperdrive.txt

[2024-05-19T18:46:25.145692][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2024-05-19T18:46:25.5639100Z][SCHEDULER][INFO]Scheduling job, id='HD_2f3a8400-3c2e-45b4-a870-764a376c95e0_0' 
[2024-05-19T18:46:25.8124205Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_2f3a8400-3c2e-45b4-a870-764a376c95e0_0' 
[2024-05-19T18:46:25.8183962Z][SCHEDULER][INFO]Scheduling job, id='HD_2f3a8400-3c2e-45b4-a870-764a376c95e0_2' 
[2024-05-19T18:46:25.7033536Z][SCHEDULER][INFO]Scheduling job, id='HD_2f3a8400-3c2e-45b4-a870-764a376c95e0_1' 
[2024-05-19T18:46:25.9553421Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_2f3a8400-3c2e-45b4-a870-764a376c95e0_1' 
[20

{'runId': 'HD_2f3a8400-3c2e-45b4-a870-764a376c95e0',
 'target': 'demo-cluster',
 'status': 'Completed',
 'startTimeUtc': '2024-05-19T18:46:24.381137Z',
 'endTimeUtc': '2024-05-19T18:55:00.591988Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '421c2401-d340-4720-a410-46351dedfe3e',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1040-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.56.0',
  'space_size': 'infinite_space_size',
  'best_child_run_id': 'HD_2f3a8400-3c2e-45b4-a870-764a376c95e0_12',
  'score': '0.912600145666424',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_2f3a8400-3c2e-45b4-a870-764a376c95e0_12'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryVa

In [16]:
# Submit your automl run

### YOUR CODE HERE ###
correct_run = hyperdrive_run.get_best_run_by_primary_metric()
correct_run


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,HD_2f3a8400-3c2e-45b4-a870-764a376c95e0_12,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [17]:

import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
correct_run.register_model(model_name="best-model", model_path=os.path.join("outputs", "best-model.pkl"))

ModelPathNotFoundException: ModelPathNotFoundException:
	Message: Could not locate the provided model_path outputs/best-model.pkl in the set of files uploaded to the run: ['logs/azureml/dataprep/0/backgroundProcess.log', 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/0/rslex.log.2024-05-19-18', 'outputs/model.pkl', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot_capability/snapshot-capability.log', 'user_logs/std_log.txt']
                See https://aka.ms/run-logging for more details.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Could not locate the provided model_path outputs/best-model.pkl in the set of files uploaded to the run: ['logs/azureml/dataprep/0/backgroundProcess.log', 'logs/azureml/da

['logs/azureml/dataprep/0/backgroundProcess.log',
 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/0/rslex.log.2024-05-19-18',
 'outputs/model.pkl',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'system_logs/snapshot_capability/snapshot-capability.log',
 'user_logs/std_log.txt']

In [ ]:
clusterhandler.delete()

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
correct_run.get_file_names()